# Deaths by age and booster status

#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
import numpy as np

/Users/stiles/.local/share/virtualenvs/grid-notebooks-Amv9DX-2/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
today = pd.to_datetime("today").strftime("%Y-%m-%d")

---

## Read [CDC data](https://data.cdc.gov/Public-Health-Surveillance/Rates-of-COVID-19-Cases-or-Deaths-by-Age-Group-and/d6p8-wqjm) found [here](https://covid.cdc.gov/covid-data-tracker/#rates-by-vaccine-status)

#### Rates of COVID-19 Cases or Deaths by Age Group and Vaccination Status and Second Booster Dose

In [132]:
url = "https://data.cdc.gov/api/views/d6p8-wqjm/rows.csv?accessType=DOWNLOAD"

In [143]:
src = pd.read_csv(url, dtype={"mmwr_week": str})

In [145]:
src.columns = src.columns.str.lower().str.replace(" ", "_", regex=False)

In [146]:
src["week_no"] = src["mmwr_week"].str[4:].astype(str)
src["year"] = src["mmwr_week"].str[:4].astype(str)
src["week"] = src["year"] + "-W" + src["week_no"]

In [147]:
src.head()

,outcome,month,mmwr_week,age_group,vaccine_product,boosted_with_outcome,boosted_population,primary_series_only_with_outcome,primary_series_only_population,unvaccinated_with_outcome,unvaccinated_population,crude_booster_ir,crude_primary_series_only_ir,crude_unvax_ir,crude_booster_irr,crude_irr,age_adj_booster_ir,age_adj_vax_ir,age_adj_unvax_ir,age_adj_booster_irr,age_adj_irr,continuity_correction,week_no,year,week
0,case,SEP 2021,202138,18-49,all_types,127,331084,50180,53236513,167028,36028226.00,38.358845,94.258615,463.603176,12.085952,4.918417,NaN,NaN,NaN,NaN,NaN,0,38,2021,2021-W38
1,case,SEP 2021,202138,50-64,all_types,154,426290,27016,30594270,45797,10071669.00,36.125642,88.304117,454.711131,12.586936,5.149376,NaN,NaN,NaN,NaN,NaN,0,38,2021,2021-W38
2,case,SEP 2021,202138,65+,all_types,282,1001538,26318,29727535,23675,5011027.55,28.156695,88.530717,472.457989,16.779597,5.336656,NaN,NaN,NaN,NaN,NaN,1,38,2021,2021-W38
3,case,SEP 2021,202138,all_ages_adj,Janssen,6,31032,10395,9720687,236500,51110922.55,19.334880,106.936886,462.719098,23.931832,4.327030,21.379326,106.071635,463.314336,21.671139,4.367938,0,38,2021,2021-W38
4,case,SEP 2021,202138,all_ages_adj,Moderna,216,772170,33527,44947832,236500,51110922.55,27.973115,74.590917,462.719098,16.541565,6.203424,35.142076,76.365838,463.314336,13.184034,6.067037,0,38,2021,2021-W38


In [148]:
src["week_date"] = pd.to_datetime(
    src["year"].astype(str) + src["week_no"].astype(str) + "1", format="%G%V%w"
)

In [149]:
src.columns

Index(['outcome', 'month', 'mmwr_week', 'age_group', 'vaccine_product',
       'boosted_with_outcome', 'boosted_population',
       'primary_series_only_with_outcome', 'primary_series_only_population',
       'unvaccinated_with_outcome', 'unvaccinated_population',
       'crude_booster_ir', 'crude_primary_series_only_ir', 'crude_unvax_ir',
       'crude_booster_irr', 'crude_irr', 'age_adj_booster_ir',
       'age_adj_vax_ir', 'age_adj_unvax_ir', 'age_adj_booster_irr',
       'age_adj_irr', 'continuity_correction', 'week_no', 'year', 'week',
       'week_date'],
      dtype='object')

In [155]:
src_slim = src[
    (src["vaccine_product"] == "all_types") & (src["age_group"] != "all_ages_adj")
][
    [
        "outcome",
        "age_group",
        "crude_booster_ir",
        "crude_primary_series_only_ir",
        "crude_unvax_ir",
        "week_date",
    ]
].copy()

In [156]:
src_slim

,outcome,age_group,crude_booster_ir,crude_primary_series_only_ir,crude_unvax_ir,week_date
0,case,18-49,38.358845,94.258615,463.603176,2021-09-20
1,case,50-64,36.125642,88.304117,454.711131,2021-09-20
2,case,65+,28.156695,88.530717,472.457989,2021-09-20
7,case,18-49,34.976923,88.458604,399.501516,2021-09-27
8,case,50-64,32.472461,84.483240,405.424627,2021-09-27
9,case,65+,25.430181,85.016760,442.176743,2021-09-27
14,case,18-49,39.089221,81.910686,355.892684,2021-10-04
15,case,50-64,28.069723,79.365161,369.313966,2021-10-04
16,case,65+,23.114820,80.703691,415.042203,2021-10-04
21,case,18-49,26.684930,73.634315,306.218807,2021-10-11
